In [13]:
import pandas as pd
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

Загружаем предобработаные комментарии

In [14]:
train_data = pd.read_csv("../datasets/train_clean_data.csv")
test_data = pd.read_csv("../datasets/test_clean_data.csv")

In [15]:
train_data.shape, test_data.shape

((25000, 4), (25000, 4))

Для получение embedding-ов используем модель ```intfloat/multilingual-e5-large```

In [16]:
MODEL_NAME = 'intfloat/multilingual-e5-large'

In [17]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)

Ф-ция для генерации embedding-ов из текста

In [18]:
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().tolist()

Генерируем embeddings побатчево

In [19]:
BATCH_SIZE = 32 # Размер батча

In [ ]:
embeddings = []

for i in tqdm(range(0, len(train_data), BATCH_SIZE), desc="[train_data] Получение embedding-ов из комментариев"):
    batch = train_data['clean_text'].iloc[i:i + BATCH_SIZE].tolist()
    embeddings.extend(get_embeddings(batch))

train_data['text_embedding'] = embeddings

In [ ]:
embeddings = []

for i in tqdm(range(0, len(test_data), BATCH_SIZE), desc="[test_data] Получение embedding-ов из комментариев"):
    batch = test_data['clean_text'].iloc[i:i + BATCH_SIZE].tolist()
    embeddings.extend(get_embeddings(batch))

test_data['text_embedding'] = embeddings

In [ ]:
train_data[['text_embedding', 'ratings']].to_csv("../datasets/train_with_embeddings.csv", index=False)
test_data[['text_embedding', 'ratings']].to_csv("../datasets/test_with_embeddings.csv", index=False)